In [ ]:
import os

import gym
import numpy as np
import pandas as pd
import seaborn as sns

from ray import init, rllib, tune, shutdown

In [ ]:
from attack_simulator.agents import DEFENDERS
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [ ]:
class AgentPolicy(rllib.policy.Policy):
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        agent_config = dict(
            input_dim=observation_space.shape[0],
            num_actions=action_space.n,
            random_seed=config["seed"],
            attack_graph=config["env_config"]["attack_graph"],
        )
        self._agent = DEFENDERS[config["agent_type"]](agent_config)

    def compute_actions(self, observations, *args, **kwargs):
        return [self._agent.act(obs) for obs in observations], [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass


def template_agent(agent_type):
    default_config = rllib.agents.trainer.with_common_config(dict(agent_type=agent_type))
    return rllib.agents.trainer_template.build_trainer(
        name=agent_type,
        default_policy=AgentPolicy,
        default_config=default_config,
    )

In [ ]:
from copy import deepcopy


class AlphaZeroWrapper(gym.Env):
    def __init__(self, config):
        self.env = config["env_class"](config)
        self.action_space = self.env.action_space
        assert isinstance(
            self.action_space, gym.spaces.Discrete
        ), "AlphaZero requires a Discrete action space"
        shape = (self.action_space.n,)
        self.observation_space = gym.spaces.Dict(
            dict(obs=self.env.observation_space, action_mask=gym.spaces.Box(0, 1, shape))
        )
        self.reward = 0
        self.mask = np.full(shape, 1, dtype="int8")

    def reset(self):
        self.reward = 0
        observation = self.env.reset()
        return dict(obs=observation, action_mask=self.mask)

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.reward += reward
        reward = self.reward if done else 0
        return dict(obs=observation, action_mask=self.mask), reward, done, info

    def set_state(self, state):
        env, self.reward = state
        self.env = deepcopy(env)
        return dict(obs=self.env.observation, action_mask=self.mask)

    def get_state(self):
        return deepcopy(self.env), self.reward

    def close(self):
        self.env.close()

    def render(self, mode=None):
        self.env.render(mode)

    def seed(self, seed=None):
        self.env.seed(seed)

In [ ]:
# 'contrib/AlphaZero' does NOT appear to work without its custom dense model
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel

rllib.models.ModelCatalog.register_custom_model("alpha_zero_dense_model", DenseModel)

In [ ]:
if os.path.isdir("/var/run/secrets/kubernetes.io"):  # inside k8s pod
    init(address="auto")
else:
    # listen on all interfaces inside a container for port-forwarding to work
    dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
    init(num_cpus=4, dashboard_host=dashboard_host)

# ALTERNATIVE: use the "Ray client" to connect to a remote cluster
# Unfortunately, JupyterNotebookReporter displays an object reference
# <IPython.core.display.HTML object> instead of content...
# --- --- ---
#
# from ray.util.client import worker
#
# worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1
#
# ray_client_server = 'host.docker.internal' if os.path.exists("/.dockerenv") else '127.0.0.1'
# try:
#     init(address=f'ray://{ray_client_server}:10001')
# except Connection Error:
#     pass  # TODO: try something else...

In [ ]:
agent_types = ["R2D2", "contrib/AlphaZero", "rule-based", "random"]
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
seeds = [0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]
iterations = 10
rollouts = 10

common_config = dict(
    # log_level='DEBUG',
    framework="torch",
    env=AttackSimulationEnv,
    env_config=dict(attack_graph=tune.grid_search(graphs)),
    seed=tune.grid_search(seeds),
    # common evaluation settings
    evaluation_num_workers=1,
    evaluation_config=dict(explore=False),
    evaluation_num_episodes=rollouts,
)
train_and_eval_config = dict(
    common_config,
    model=dict(use_lstm=True),
    num_workers=8,
    # evaluation at the end
    evaluation_interval=iterations,
)
eval_only_config = dict(
    common_config,
    # evaluation ONLY: avoid MultiGPU optimizer, set all relevant sizes to 0
    simple_optimizer=True,
    num_workers=0,
    train_batch_size=0,
    rollout_fragment_length=0,
    timesteps_per_iteration=0,
    # evaluation at the end
    evaluation_interval=1,
)

frames = []
for agent_type in agent_types:
    if agent_type in DEFENDERS:
        agent = template_agent(agent_type)
        config = eval_only_config
        stop = dict(training_iteration=0)
    else:
        agent = agent_type
        config = train_and_eval_config
        if agent_type == "contrib/AlphaZero":
            config["env_config"].update(env_class=config["env"])
            config.update(env=AlphaZeroWrapper, model=dict(custom_model="alpha_zero_dense_model"))
        stop = dict(training_iteration=iterations)  # TODO: additional stopping criteria?

    config["env_config"].update(agent_type=agent_type)
    results = tune.run(
        agent,
        config=config,
        stop=stop,
        max_failures=3,
        queue_trials=True,
        raise_on_failed_trial=False,
        progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
    )
    frames.append(results.result_df)

df = pd.concat(frames, ignore_index=True)

In [ ]:
shutdown()

In [ ]:
rename = {
    "config.env_config.attack_graph": "graph",
    "config.env_config.agent_type": "Agent",
    "evaluation.hist_stats.episode_reward": "returns",
    "evaluation.hist_stats.episode_lengths": "lengths",
}

df = df[rename.keys()].rename(columns=rename)
df["Graph size"] = df["graph"].apply(lambda g: g.num_attacks)
del df["graph"]
df["tuple"] = df.apply(lambda t: list(zip(t.returns, t.lengths)), axis="columns")
del df["returns"]
del df["lengths"]
df = df.explode("tuple", ignore_index=True)
df[["Returns", "Episode lengths"]] = df["tuple"].tolist()
del df["tuple"]
df

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Returns vs Size (random attacker)")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Episode lengths vs Size (random attacker)")

In [ ]:
pd.set_option("display.max_columns", 32)
df.groupby("Agent").describe()